# Sweeps notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pprint
import os
import re
import json
import sys
import wandb
from model import *
from pytorch_lightning.loggers import WandbLogger
import tensorflow

c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
# Define sweep config
sweep_config = {
    'method': 'random' # random, grid, or bayes
    }

In [3]:
# Define metric
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   # minimize or maximize
    }
# add in sweep_config
sweep_config['metric'] = metric

In [4]:
epochs = 10
count = 3
architecture = 'CNN'
dataset = 'Poems'
# Define hyperparameter space
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'embedding_dim': {
        'values': [128, 256, 512]
        },
    }
# we can indicate the distribution for continuous variables
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.1
      }
    })
# we set values that we want to track but don't want to change, just indicate 1 value
parameters_dict.update({
    'epochs': {'value': epochs},
    "architecture":{'value': architecture},
    "dataset": {'value': dataset},        
    })
# add params in sweep_config
sweep_config['parameters'] = parameters_dict

In [5]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'architecture': {'value': 'CNN'},
                'dataset': {'value': 'Poems'},
                'embedding_dim': {'values': [128, 256, 512]},
                'epochs': {'value': 10},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [6]:
# Log in to weights and biases
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jjdsantiago3 (msds_mlops2023_lt2). Use `wandb login --relogin` to force relogin


True

In [7]:
sweep_id = wandb.sweep(sweep_config, project="mlo-final-project")

Create sweep with ID: su7b1vnn
Sweep URL: https://wandb.ai/msds_mlops2023_lt2/mlo-final-project/sweeps/su7b1vnn


In [8]:
# define hyperparam search function
def hyperparam_search(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        # use config params
        learning_rate = config.learning_rate
        optimizer = config.optimizer
        embedding_dim = config.embedding_dim
        epochs = config.epochs

        # define model
        _model = poem_classifier_model()

        _model.load_data()
        _model.preprocess()
        _model.train(embd_dim=embedding_dim, epochs=epochs, lr=learning_rate, optimizer=optimizer)

        # print(_model.trained_model.history)

        for i in range(epochs):
            wandb.log({"train_loss": _model.trained_model.history['loss'][i],
                       "train_acc": _model.trained_model.history['acc'][i],
                       "val_loss": _model.trained_model.history['val_loss'][i], 
                       "val_acc": _model.trained_model.history['val_acc'][i]})
        
        results = _model.test()
        wandb.log({"test_loss": results[0], "test_acc": results[1]})

wandb.agent(sweep_id, hyperparam_search, count=count) # count - num iters

wandb: Agent Starting Run: iucm24k0 with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 128
wandb: 	epochs: 10
wandb: 	learning_rate: 0.05715589243162577
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

[[  0   0   0 ...   5   2 129]
 [  0   0   0 ... 247 398   2]
 [  0   0   0 ...   1 109  63]
 ...
 [  0   0   0 ...  29   1   2]
 [  0   0   0 ... 210  18   1]
 [  0   0   0 ...  19  11 361]]
tf.Tensor(
[3. 3. 3. 0. 2. 1. 2. 1. 2. 1. 2. 2. 3. 1. 3. 1. 1. 3. 3. 2. 0. 2. 3. 3.
 1. 2. 2. 0. 2. 3. 2. 2. 3. 3. 3. 1. 3. 3. 1. 1. 3. 3. 1. 2. 1. 3. 1. 0.
 3. 0. 1. 1. 0. 3. 1. 0. 2. 0. 1. 3. 2. 3. 2. 1. 1. 2. 2. 2. 1. 1. 3. 2.
 1. 3. 2. 3. 1. 2. 3. 3. 1. 3. 1. 2. 1. 3. 1. 0. 2. 3. 1. 3. 1. 0. 2. 3.
 3. 1. 2. 0. 3. 0. 2. 0. 1. 1. 2. 2. 1. 3. 1. 1. 2. 1. 1. 3. 2. 0. 2. 3.
 1. 1. 1. 2. 2. 0. 1. 0. 1. 0. 1. 1. 2. 1. 1. 3. 1. 1. 3. 2. 2. 2. 2. 2.
 0. 3. 2. 0. 2. 3. 0. 2. 3. 1. 3. 2. 3. 2. 1. 1. 1. 1. 2. 3. 2. 3. 3. 1.
 2. 3. 1. 2. 1. 1. 3. 2. 2. 0. 2. 0. 0. 0. 3. 3. 0. 2. 1. 1. 2. 1. 1. 1.
 2. 2. 1. 1. 3. 3. 2. 2. 2. 2. 2. 0. 1. 0. 2. 0. 3. 2. 1. 3. 3. 2. 2. 3.
 3. 0. 3. 3. 3. 0. 1. 1. 0. 2. 2. 1. 2. 3. 2. 2. 0. 2. 3. 3. 1. 0. 2. 1.
 1. 0. 0. 3. 1. 1. 0. 1. 1. 1. 0. 0. 3. 3. 1. 1. 1. 3. 2. 2. 3. 3. 

Traceback (most recent call last):
  File "C:\Users\Jesli's Laptop\AppData\Local\Temp\ipykernel_32032\3043076251.py", line 19, in hyperparam_search
    _model.train(embd_dim=embedding_dim, epochs=epochs, lr=learning_rate, optimizer=optimizer)
  File "c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py", line 113, in train
    self.trained_model = self.model.fit(inpt_tr, otpt_tr, epochs=epochs, validation_data=(inpt_ts, otpt_ts))
  File "c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\JESLI'~1\AppData\Local\Temp\__autograph_generated_filegr136j1a.py", line 15, in tf__train_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\k

Run iucm24k0 errored: ValueError('in user code:\n\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1338, in train_function  *\n        return step_function(self, iterator)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1322, in step_function  **\n        outputs = model.distribute_strategy.run(run_step, args=(data,))\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1303, in run_step  **\n        outputs = model.train_step(data)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1085, in train_step\n        return self.compute_metrics(x, y, y_pred, sample_weight)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Prog

...

c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

Run n6xkkqfp errored: ValueError('in user code:\n\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1338, in train_function  *\n        return step_function(self, iterator)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1322, in step_function  **\n        outputs = model.distribute_strategy.run(run_step, args=(data,))\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1303, in run_step  **\n        outputs = model.train_step(data)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1085, in train_step\n        return self.compute_metrics(x, y, y_pred, sample_weight)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Prog

[[  0   0   0 ...   1   1   1]
 [  0   0   0 ...   1   1   1]
 [  0   0   0 ... 181   7   1]
 ...
 [  0   0   0 ... 122   1  29]
 [  0   0   0 ...  79  11 218]
 [  0   0   0 ...   2 383   3]]
tf.Tensor(
[1. 1. 2. 2. 1. 1. 2. 0. 1. 2. 2. 1. 1. 3. 1. 1. 3. 3. 3. 2. 3. 2. 2. 2.
 2. 1. 1. 1. 3. 1. 0. 1. 3. 1. 3. 2. 2. 3. 3. 3. 0. 0. 2. 0. 0. 1. 0. 3.
 0. 3. 1. 1. 2. 1. 3. 2. 1. 3. 1. 3. 3. 0. 3. 1. 3. 3. 2. 0. 1. 1. 3. 0.
 1. 2. 2. 1. 2. 3. 2. 3. 3. 0. 2. 0. 1. 0. 0. 2. 3. 1. 3. 0. 0. 1. 0. 3.
 3. 1. 3. 3. 0. 2. 2. 2. 1. 1. 3. 0. 1. 1. 1. 2. 2. 3. 2. 2. 0. 3. 2. 2.
 2. 1. 2. 2. 3. 1. 1. 1. 2. 0. 3. 2. 1. 3. 3. 0. 3. 0. 1. 2. 3. 3. 2. 3.
 1. 2. 2. 1. 3. 2. 2. 3. 1. 0. 0. 3. 3. 3. 2. 2. 0. 3. 2. 3. 3. 2. 3. 3.
 3. 2. 3. 2. 3. 0. 2. 3. 1. 1. 2. 2. 0. 0. 0. 3. 2. 1. 3. 2. 1. 0. 0. 1.
 2. 0. 3. 1. 1. 1. 3. 1. 1. 2. 3. 0. 1. 2. 0. 2. 2. 2. 3. 3. 1. 2. 1. 0.
 2. 1. 0. 1. 1. 2. 3. 1. 0. 1. 0. 3. 3. 1. 2. 3. 1. 3. 2. 3. 3. 2. 3. 0.
 3. 3. 2. 3. 3. 3. 1. 2. 3. 2. 1. 3. 3. 0. 1. 1. 1. 3. 3. 0. 3. 2. 

c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:50: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

Run sz7ffjai errored: ValueError('in user code:\n\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1338, in train_function  *\n        return step_function(self, iterator)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1322, in step_function  **\n        outputs = model.distribute_strategy.run(run_step, args=(data,))\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1303, in run_step  **\n        outputs = model.train_step(data)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\keras\\src\\engine\\training.py", line 1085, in train_step\n        return self.compute_metrics(x, y, y_pred, sample_weight)\n    File "c:\\Users\\Jesli\'s Laptop\\AppData\\Local\\Prog

In [9]:
wandb.finish()